### Student Management System

In [ ]:
import os
from statistics import median

DATA_FILE = "students.txt"

#Helper Input/Output

def read_students():
    """Read students.txt and return a list of dicts.
    Handles missing/invalid lines gracefully (skips them)."""
    students = []
    if not os.path.exists(DATA_FILE):
        return students
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            parts = line.split(",")
            if len(parts) != 5:
                
#Skip invalid line but InFORM in console (helpful when debugging)
                
                print(f"[warning] skipping malformed line {i} in {DATA_FILE}: {line}")
                continue
            sid, name, age, grade, marks = parts
            try:
                student = {
                    "id": int(sid),
                    "name": name.strip(),
                    "age": int(age),
                    "grade": grade.strip().upper(),
                    "marks": int(marks)
                }
            except ValueError:
                print(f"[warning] invalid data types on line {i}, skipping: {line}")
                continue
            students.append(student)
    return students

def write_students(students):
    """Overwrite students.txt with the list of student dicts."""
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        for s in students:
            line = f"{s['id']},{s['name']},{s['age']},{s['grade']},{s['marks']}\n"
            f.write(line)

#Input validation helpers

def input_int(prompt, min_val=None, max_val=None):
    while True:
        val = input(prompt).strip()
        if not val:
            print("Input cannot be empty. Try again.")
            continue
        if not val.lstrip("-").isdigit():
            print("Please enter a valid integer.")
            continue
        num = int(val)
        if min_val is not None and num < min_val:
            print(f"Value must be >= {min_val}.")
            continue
        if max_val is not None and num > max_val:
            print(f"Value must be <= {max_val}.")
            continue
        return num

def input_nonempty_string(prompt):
    while True:
        s = input(prompt).strip()
        if not s:
            print("Input cannot be empty.")
            continue
        #ensure not purely digits
        if s.isdigit():
            print("Name cannot be only numbers.")
            continue
        return s

def input_grade(prompt):
    while True:
        g = input(prompt).strip().upper()
        if len(g) != 1 or not g.isalpha():
            print("Grade must be a single letter (A-F).")
            continue
        if g < 'A' or g > 'F':
            print("Grade must be between A and F.")
            continue
        return g

#CRUD functions

def is_unique_id(students, sid):
    return all(s['id'] != sid for s in students)

def add_student():
    students = read_students()
    print("\n--- Add New Student ---")
    while True:
        sid = input_int("Enter ID (integer): ", min_val=0)
        if not is_unique_id(students, sid):
            print("ID already exists. Try a different ID.")
            continue
        break
    name = input_nonempty_string("Enter full name: ")
    age = input_int("Enter age: ", min_val=1)
    grade = input_grade("Enter grade (A-F): ")
    marks = input_int("Enter marks (0-100): ", min_val=0, max_val=100)

    students.append({"id": sid, "name": name, "age": age, "grade": grade, "marks": marks})
    
#append to file to avoid rewriting everything (but it's ok either way)
    
    with open(DATA_FILE, "a", encoding="utf-8") as f:
        f.write(f"{sid},{name},{age},{grade},{marks}\n")
    print(f"Student {name} added successfully.")

def view_students():
    students = read_students()
    if not students:
        print("\nNo records found.")
        return
    print("\n--- All Students ---")
    print(f"{'ID':<6} {'Name':<25} {'Age':<4} {'Grade':<6} {'Marks':<6}")
    print("-"*50)
    for s in students:
        print(f"{s['id']:<6} {s['name']:<25} {s['age']:<4} {s['grade']:<6} {s['marks']:<6}")
    print("-"*50)
    print(f"Total records: {len(students)}")

def search_student():
    students = read_students()
    if not students:
        print("\nNo records to search.")
        return
    print("\n--- Search Student ---")
    choice = input("Search by (1) ID or (2) Name? [1/2]: ").strip()
    if choice == "1":
        sid = input_int("Enter ID to search: ", min_val=0)
        found = [s for s in students if s['id'] == sid]
    else:
        q = input("Enter full or partial name: ").strip().lower()
        found = [s for s in students if q in s['name'].lower()]
    if not found:
        print("No matching students found.")
        return
    print(f"Found {len(found)} record(s):")
    for s in found:
        print(f"ID: {s['id']} | Name: {s['name']} | Age: {s['age']} | Grade: {s['grade']} | Marks: {s['marks']}")

def update_student():
    students = read_students()
    if not students:
        print("\nNo records to update.")
        return
    print("\n--- Update Student ---")
    sid = input_int("Enter ID of student to update: ", min_val=0)
    idx = next((i for i, s in enumerate(students) if s['id'] == sid), None)
    if idx is None:
        print("Student ID not found.")
        return
    s = students[idx]
    print(f"Current record: {s}")
    
#Selective updates
    
    if input("Update name? (y/N): ").strip().lower() == 'y':
        s['name'] = input_nonempty_string("New name: ")
    if input("Update age? (y/N): ").strip().lower() == 'y':
        s['age'] = input_int("New age: ", min_val=1)
    if input("Update grade? (y/N): ").strip().lower() == 'y':
        s['grade'] = input_grade("New grade (A-F): ")
    if input("Update marks? (y/N): ").strip().lower() == 'y':
        s['marks'] = input_int("New marks (0-100): ", min_val=0, max_val=100)
        
#optionally allow ID change (but ensure uniqueness)
    
    if input("Change ID? (y/N): ").strip().lower() == 'y':
        while True:
            new_id = input_int("New ID: ", min_val=0)
            if new_id == sid or is_unique_id(students, new_id):
                s['id'] = new_id
                break
            else:
                print("That ID already exists. Pick another.")
    students[idx] = s
    write_students(students)
    print("Record updated.")

def delete_student():
    students = read_students()
    if not students:
        print("\nNo records to delete.")
        return
    print("\n--- Delete Student ---")
    sid = input_int("Enter ID to delete: ", min_val=0)
    found = [s for s in students if s['id'] == sid]
    if not found:
        print("No such student.")
        return
    s = found[0]
    confirm = input(f"Are you sure you want to delete {s['name']} (ID {s['id']})? (y/N): ").strip().lower()
    if confirm == 'y':
        students = [x for x in students if x['id'] != sid]
        write_students(students)
        print("Record deleted.")
    else:
        print("Deletion cancelled.")

# Analysis functions

def analyze_data():
    students = read_students()
    if not students:
        print("\nNo data to analyze.")
        return
    marks_list = [s['marks'] for s in students]
    avg_marks = sum(marks_list) / len(marks_list)
    top = max(students, key=lambda x: x['marks'])
    bottom = min(students, key=lambda x: x['marks'])
    below_avg = [s for s in students if s['marks'] < avg_marks]
    
#Custom insight 1: grade distribution
    
    grade_dist = {}
    for s in students:
        grade_dist[s['grade']] = grade_dist.get(s['grade'], 0) + 1
    
#Custom insight 2: median marks and pass-rate (assuming pass >= 40)
    
    med = median(marks_list)
    passed = sum(1 for m in marks_list if m >= 40)
    pass_rate = (passed / len(students)) * 100

    print("\n--- Data Analysis ---")
    print(f"Average Marks: {avg_marks:.2f}")
    print(f"Median Marks: {med}")
    print(f"Top Performer: {top['name']} (ID {top['id']}) - {top['marks']}")
    print(f"Lowest Marks: {bottom['name']} (ID {bottom['id']}) - {bottom['marks']}")
    print(f"Students Below Average: {len(below_avg)}")
    print(f"Pass Rate (>=40): {pass_rate:.1f}% ({passed}/{len(students)})")
    print("\nGrade Distribution:")
    for grade in sorted(grade_dist.keys()):
        print(f" Grade {grade}: {grade_dist[grade]} student(s)")
        
#Show a small list of underperformers (custom)
    
    under = sorted(below_avg, key=lambda x: x['marks'])[:5]  #up to 5 lowest below average
    if under:
        print("\nUnderperformers (below average) - listing up to 5:")
        for u in under:
            print(f" {u['name']} (ID {u['id']}) - {u['marks']} marks")
            
#Simple summary insight
    
    if avg_marks >= 85:
        print("\nInsight: Excellent class performance overall!")
    elif avg_marks >= 60:
        print("\nInsight: Decent results, consider targeted help for weaker students.")
    else:
        print("\nInsight: Many students are struggling — consider intervention classes.")

#Utility (nice-to-have) 

def export_csv(filename="students_export.csv"):
    students = read_students()
    if not students:
        print("No data to export.")
        return
    with open(filename, "w", encoding="utf-8") as f:
        f.write("id,name,age,grade,marks\n")
        for s in students:
            f.write(f"{s['id']},{s['name']},{s['age']},{s['grade']},{s['marks']}\n")
    print(f"Exported {len(students)} records to {filename}")

#Main MEnu

def main():
    while True:
        print("\n====== Smart Student Record Analyzer ======")
        print("1. Add Student")
        print("2. View All Students")
        print("3. Search Student")
        print("4. Update Student")
        print("5. Delete Student")
        print("6. Analyze Data")
        print("7. Export CSV (extra)")
        print("8. Exit")
        choice = input("Enter your choice: ").strip()
        if choice == "1":
            add_student()
        elif choice == "2":
            view_students()
        elif choice == "3":
            search_student()
        elif choice == "4":
            update_student()
        elif choice == "5":
            delete_student()
        elif choice == "6":
            analyze_data()
        elif choice == "7":
            export_csv()
        elif choice == "8":
            print("Thanks for using Smart Student Record Analyzer. Bye!")
            break
        else:
            print("Invalid choice. Enter number from 1 to 8.")

if __name__ == "__main__":
    main()



====== Smart Student Record Analyzer ======
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Export CSV (extra)
8. Exit


Enter your choice:  2



--- All Students ---
ID     Name                      Age  Grade  Marks 
--------------------------------------------------
1      Sooraj Hamirani           24   A      87    
2      Daniyal                   28   A      89    
3      Aijaz                     34   B      67    
4      M. Talha                  27   C      57    
5      M. Ali                    24   B      76    
--------------------------------------------------
Total records: 5

====== Smart Student Record Analyzer ======
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Export CSV (extra)
8. Exit


Enter your choice:  7


Exported 5 records to students_export.csv

====== Smart Student Record Analyzer ======
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Export CSV (extra)
8. Exit


...


--- Update Student ---


Enter ID of student to update:  1


Current record: {'id': 1, 'name': 'Sooraj Hamirani', 'age': 24, 'grade': 'A', 'marks': 87}


Update name? (y/N):  y
New name:  Sooraj
Update age? (y/N):  n
Update grade? (y/N):  n
Update marks? (y/N):  n
Change ID? (y/N):  n


Record updated.

====== Smart Student Record Analyzer ======
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Export CSV (extra)
8. Exit
